In [1]:
import pandas as pd

# ---------------------------------------------------------
# Función: download_csv_from_public_s3
# ---------------------------------------------------------
def download_csv_from_public_s3(bucket_name: str, file_key: str, sep: str = None) -> pd.DataFrame:
    url = f"https://{bucket_name}.s3.amazonaws.com/{file_key}"
    return pd.read_csv(
        url,
        sep=sep,
        engine="python",
        quotechar='"',
        escapechar="\\",
        on_bad_lines="skip"
    )

In [2]:
# Cargar datasets desde el bucket público a memoria
df_netflix = download_csv_from_public_s3("desafio-rkd", "netflix_titles.csv", sep=";")
df_disney = download_csv_from_public_s3("desafio-rkd", "disney_plus_titles.csv", sep=",")

# Agregar columna de plataforma
df_netflix["platform"] = "Netflix"
df_disney["platform"] = "Disney+"

print("Netflix:", df_netflix.shape)
print("Disney+: ", df_disney.shape)


Netflix: (8809, 13)
Disney+:  (1450, 13)


In [3]:
for df, name in [(df_netflix, "Netflix"), (df_disney, "Disney+")]:
    print(f"\n--- {name} ---")
    print(df.info())
    print(df.head())



--- Netflix ---
<class 'pandas.DataFrame'>
RangeIndex: 8809 entries, 0 to 8808
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   show_id       8809 non-null   str  
 1   type          8808 non-null   str  
 2   title         8807 non-null   str  
 3   director      6173 non-null   str  
 4   cast          7983 non-null   str  
 5   country       7976 non-null   str  
 6   date_added    8797 non-null   str  
 7   release_year  8807 non-null   str  
 8   rating        8803 non-null   str  
 9   duration      8804 non-null   str  
 10  listed_in     8806 non-null   str  
 11  description   8806 non-null   str  
 12  platform      8809 non-null   str  
dtypes: str(13)
memory usage: 894.8 KB
None
  show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water              NaN   
2      s3  TV Show              Ganglands 

In [ ]:
# Valores nulos
print("Nulos Netflix:\n", df_netflix.isnull().sum())
print("Nulos Disney+:\n", df_disney.isnull().sum())

# Duplicados
print("Duplicados Netflix:", df_netflix.duplicated().sum())
print("Duplicados Disney+: ", df_disney.duplicated().sum())


Nulos Netflix:
 show_id            0
type               1
title              2
director        2636
cast             826
country          833
date_added        12
release_year       2
rating             6
duration           5
listed_in          3
description        3
platform           0
dtype: int64
Nulos Disney+:
 show_id           0
type              0
title             0
director        473
cast            190
country         219
date_added        3
release_year      0
rating            3
duration          0
listed_in         0
description       0
platform          0
dtype: int64
Duplicados Netflix: 0
Duplicados Disney+:  0


# Análisis Descriptivo y Control de Calidad de Datos

## 1. Carga de datasets
Se descargaron los datasets desde un bucket público de S3:
- **Netflix**: 8.809 registros y 13 columnas.
- **Disney+**: 1.450 registros y 13 columnas.

Se agregó una columna adicional `platform` para identificar la fuente de cada registro.

---

## 2. Exploración inicial
El dataset de **Netflix** contiene información sobre películas y series con las siguientes columnas:
- `show_id`, `type`, `title`, `director`, `cast`, `country`, `date_added`, `release_year`, `rating`, `duration`, `listed_in`, `description`, `platform`.

El dataset de **Disney+** presenta la misma estructura.

---

## 3. Valores nulos y duplicados
- **Netflix**:
  - `director`: 2.636 nulos.
  - `cast`: 826 nulos.
  - `country`: 833 nulos.
  - `date_added`: 12 nulos.
  - `rating`: 6 nulos.
  - `duration`: 5 nulos.
  - Total de duplicados: 0. (Basado en la funcion duplicated de pandas, que compara registro vs registro de arriba, hay que revisar bien)

- **Disney+**:
  - `director`: 473 nulos.
  - `cast`: 190 nulos.
  - `country`: 219 nulos.
  - `date_added`: 3 nulos.
  - `rating`: 3 nulos.
  - Total de duplicados: 0.

**Hallazgo:** Los campos relacionados con personas (`director`, `cast`) y país (`country`) presentan una cantidad significativa de valores faltantes. Esto puede afectar análisis posteriores de participación de actores/directores o distribución geográfica.

---

# Propuestas de Solución

## 1. Manejo de valores nulos
- **Director y Cast**: imputar valores faltantes con `"Unknown"` o mantenerlos como nulos según el análisis.  
  - Ventaja: evita perder registros en consultas.  
  - Alternativa: crear una tabla auxiliar de "personas desconocidas" para mantener consistencia en el modelo relacional.  
- **Country**: normalizar y completar valores faltantes con `"Unknown"` o mediante reglas de negocio (ej. si el título es de una franquicia conocida).  
- **Date_added**: imputar con la fecha de carga aproximada si se conoce, o dejar nulo.  
- **Rating**: imputar con `"Not Rated"` para mantener consistencia.  
- **Tabla de registros incompletos**: crear una tabla auxiliar (`incomplete_records`) que almacene los registros con campos críticos faltantes. Una vez enriquecidos o completados, se reincorporan al flujo normal.

---

## 2. Normalización de columnas
- **Duración**:  
  - Extraer minutos en películas (`duration_minutes`).  
  - Crear columna adicional `duration_seasons` para series, ya que la duración en minutos no aplica.  
- **Fechas**: convertir `date_added` a tipo `datetime` para análisis temporales.  
- **Texto**: homogeneizar nombres de actores y directores a minúsculas (`.str.lower()`), eliminar espacios extra con `.strip()`.  
- **Países**: normalizar nombres de países (ej. `"United States"` vs `"USA"`) usando librerías como `pycountry`.

---

## 3. Control de duplicados
- Eliminar registros duplicados con `drop_duplicates()`.  
- En caso de títulos repetidos en distintas plataformas, mantener ambos pero diferenciados por la columna `platform`.  
- Definir reglas claras de deduplicación (ej. por `show_id` o combinación de `title + platform`).

---

## 4. Preparación para modelado
- Crear claves primarias (`show_id`) y asegurar que sean únicas.  
- Separar actores en una tabla relacional (`movie_actors`) para manejar relaciones N:M.  
- Validar que `release_year` sea numérico y consistente.  
- Implementar auditoría con columnas de trazabilidad (`created_at`, `updated_at`, `source_file`) para asegurar transparencia en el flujo.

---

## 5. Arquitectura de capas (Medallion)

```text
        ┌───────────────┐
        │     RAW       │   ← Datos originales del bucket S3
        └───────┬───────┘
                │
        ┌───────▼───────┐
        │    BRONZE     │   ← Limpieza básica: nulos, duplicados, normalización
        └───────┬───────┘
                │
        ┌───────▼───────┐
        │    SILVER     │   ← Modelado intermedio: IDs, relaciones N:M, consistencia
        └───────┬───────┘
                │
        ┌───────▼───────┐
        │     GOLD      │   ← Capa analítica: métricas, cruces, BI, ML
        └───────────────┘
```

---

## 6. Importancia del Machine Learning
- Una vez que los datos están limpios y estructurados en la capa **Gold**, se pueden aplicar modelos de **Machine Learning** para enriquecer la experiencia del usuario.  
- Ejemplos de aplicaciones:  
  - **Recomendación de contenido**: en base a los gustos y patrones de consumo de cada persona, sugerir películas o series similares.  
  - **Predicción de popularidad**: estimar qué títulos tendrán mayor éxito en determinadas regiones o segmentos.  
  - **Segmentación de usuarios**: agrupar audiencias según comportamiento para personalizar campañas de marketing.  
- Beneficio: las plataformas pueden ofrecer un catálogo más relevante, mejorar la retención de usuarios y aumentar el engagement.

---

## 7. Beneficios esperados
- Datos más limpios y consistentes para consultas SQL.  
- Posibilidad de responder preguntas como "actor más repetido en Netflix" sin sesgos por nulos o duplicados.  
- Modelo relacional más robusto y escalable para futuras integraciones.  
- Flujo transparente: los registros incompletos se gestionan aparte, evitando contaminar el modelo principal.  
- Arquitectura modular que permite crecer hacia pipelines más complejos (ej. integración con dashboards o machine learning).  
- Potencial de aplicar **ML** para recomendaciones personalizadas, aumentando el valor del análisis y la utilidad de los datos.


